<a href="https://colab.research.google.com/github/ekotyoo/Racana/blob/Machine_Learning/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633724 sha256=f266579d1a198eca9668ae1326d6d0a04d9898042c9ba6647ccefb0bac7cc430
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import surprise as sp 
import tensorflow as tf

from tensorflow import keras
from collections import deque
from surprise.model_selection import cross_validate
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.metrics import mean_squared_error



In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1Fjjh_nSMWssHE4Ncv7aURngnN5wsfJ_y/view'


# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tourism_with_id.csv')

place = pd.read_csv('tourism_with_id.csv')
print(place)


    Place_Id                     Place_Name  \
0          1              Agrowisata Satria   
1          2                     Agung Bali   
2          3  Wisata Air Panas Toya Bungkah   
3          4         Air Terjun Aling-Aling   
4          5          Air Terjun Tegenungan   
..       ...                            ...   
70        71         Teras Sawah Tegalalang   
71        72                   Tirta Gangga   
72        73                  Waterbom Bali   
73        74       Alas Harum Agro Tourism    
74        75     Wisata Agro Teba Sari Bali   

                                          Description  Weekend Holiday Price  \
0   Agrowisata Satria menawarkan 'rasa' Bali pada ...                  50000   
1   Dapatkan berbagai produk oleh-oleh khas Bali b...                      0   
2   Pemandian air panas di dekat gunung Batur (Pem...                  70000   
3   Keberadaan air terjun ini semakin mempercantik...                  20000   
4   Bosan dengan wisata alam pantai

In [ ]:
# Melihat gambaran data place

place.head()

,Place_Id,Place_Name,Description,Weekend Holiday Price,Weekday Price,Category,City,Rating,Alamat,Coordinate,Lat,Long,Gambar
0,1,Agrowisata Satria,Agrowisata Satria menawarkan 'rasa' Bali pada ...,50000,50000,Agrowisata,Gianyar,3.0,"Manukaya, Kec. Tampaksiring, Kabupaten Gianya...","-8.402228546131536, 115.32296693687739",-8.402229,115.322967,V
1,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,0,0,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali","-8.700234336021559, 115.176534407375",-8.700234,115.176534,V
2,3,Wisata Air Panas Toya Bungkah,Pemandian air panas di dekat gunung Batur (Pem...,70000,70000,Alam,Bangli,4.0,"Batur Tengah, Kec. Kintamani, Kabupaten Bangli...","-8.251298580809106, 115.39981444195475",-8.251299,115.399814,V
3,4,Air Terjun Aling-Aling,Keberadaan air terjun ini semakin mempercantik...,20000,20000,Alam,Buleleng,4.5,"Jl. Raya Desa Sambangan, Banjar, Sambangan, Ke...","-8.173073590748519, 115.10513024047802",-8.173074,115.105130,V
4,5,Air Terjun Tegenungan,Bosan dengan wisata alam pantai dan ingin meni...,20000,20000,Alam,Gianyar,4.0,"Jl. Ir. Sutami, Kemenuh, Kec. Sukawati, Kabupa...","-8.575191460385602, 115.28870189592831",-8.575191,115.288702,V


In [ ]:
place.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Place_Id               75 non-null     int64  
 1   Place_Name             75 non-null     object 
 2   Description            75 non-null     object 
 3   Weekend Holiday Price  75 non-null     int64  
 4   Weekday Price          75 non-null     int64  
 5   Category               75 non-null     object 
 6   City                   75 non-null     object 
 7   Rating                 75 non-null     float64
 8   Alamat                 75 non-null     object 
 9   Coordinate             75 non-null     object 
 10  Lat                    75 non-null     float64
 11  Long                   75 non-null     float64
 12  Gambar                 75 non-null     object 
dtypes: float64(3), int64(3), object(7)
memory usage: 7.7+ KB


RATING 

In [ ]:
rate =  'https://drive.google.com/file/d/1O8nHRBufcBe26vB8GHicaTTqAlIcUv8E/view'

# to get the id part of the file
id = rate.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tourism_rating.csv')

rating = pd.read_csv('tourism_rating.csv')
rating.head()

,User_Id,Place_Id,Nama Tempat,Place_Rating
0,1,2,Agung Bali Oleh-Oleh,3
1,1,7,Aloha Ubud Swing,5
2,1,8,Bali Bird Park,5
3,1,12,Big Garden Corner,2
4,1,13,Wanagiri Hidden Hills,5


In [ ]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User_Id       3000 non-null   int64 
 1   Place_Id      3000 non-null   int64 
 2   Nama Tempat   3000 non-null   object
 3   Place_Rating  3000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 93.9+ KB


In [ ]:
min_place_rating = 10
filter_place = (rating["Place_Id"].value_counts()>min_place_rating)
filter_place = filter_place[filter_place].index.tolist()

min_user_rating = 10
filter_user = (rating["User_Id"].value_counts()>min_user_rating)
filter_user = filter_user[filter_user].index.tolist()

data_filtered = rating[(rating["Place_Id"].isin(filter_place)) & 
                       (rating["User_Id"].isin(filter_place))]
del filter_place,filter_user,min_place_rating,min_user_rating
print('Shape User-Ratings unfiltered:\t{}'.format(rating.shape))
print('Shape User-Ratings filtered:\t{}'.format(data_filtered.shape))

Shape User-Ratings unfiltered:	(3000, 4)
Shape User-Ratings filtered:	(2250, 4)


In [ ]:
data_filtered = data_filtered.sample(frac=1).reset_index(drop=True)

n =500

df_train = data_filtered[:-n]
df_test = data_filtered[-n:]



# Matrix Factorisation With Keras And Gradient Descent

In [ ]:
# Create user- & place-id mapping
user_id_mapping = {id:i for i, id in enumerate(data_filtered['User_Id'].unique())}
place_id_mapping = {id:i for i, id in enumerate(data_filtered['Place_Id'].unique())}


# Create correctly mapped train- & testset
train_user_data = df_train['User_Id'].map(user_id_mapping)
train_place_data = df_train['Place_Id'].map(place_id_mapping)

test_user_data = df_test['User_Id'].map(user_id_mapping)
test_place_data = df_test['Place_Id'].map(place_id_mapping)


# Get input variable-sizes
users = len(user_id_mapping)
places = len(place_id_mapping)
embedding_size = 10


##### Create model

# Set input layers
user_id_input = Input(shape=[1], name='user')
place_id_input = Input(shape=[1], name='place')

# Create embedding layers for users and places
user_embedding = Embedding(output_dim=embedding_size, 
                           input_dim=users,
                           input_length=1, 
                           name='user_embedding')(user_id_input)
place_embedding = Embedding(output_dim=embedding_size, 
                            input_dim=places,
                            input_length=1, 
                            name='item_embedding')(place_id_input)

# Reshape the embedding layers
user_vector = Reshape([embedding_size])(user_embedding)
place_vector = Reshape([embedding_size])(place_embedding)

# Compute dot-product of reshaped embedding layers as prediction
y = Dot(1, normalize=False)([user_vector, place_vector])
x = Dense(1024, activation='relu')(y)
out = Dense(1, activation='sigmoid')(x)
# Setup model
model = Model(inputs=[user_id_input, place_id_input], outputs=out)
model.compile(loss='mse', optimizer='adam')


# Fit model
model.fit([train_user_data, train_place_data],
          df_train['Place_Rating'],
          batch_size=256, 
          epochs=100,
          validation_split=0.1,
          shuffle=True)

# Test model
y_pred = model.predict([test_user_data, test_place_data])
y_true = df_test['Place_Rating'].values
#  Compute RMSE
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Matrix-Factorization: {:.4f} RMSE'.format(rmse))

Epoch 1/100
7/7 [==============================] - 1s 29ms/step - loss: 14.2980 - val_loss: 13.6928
Epoch 2/100
7/7 [==============================] - 0s 7ms/step - loss: 13.9646 - val_loss: 13.3115
Epoch 3/100
7/7 [==============================] - 0s 7ms/step - loss: 13.5441 - val_loss: 12.8525
Epoch 4/100
7/7 [==============================] - 0s 8ms/step - loss: 13.0525 - val_loss: 12.3506
Epoch 5/100
7/7 [==============================] - 0s 7ms/step - loss: 12.5410 - val_loss: 11.8679
Epoch 6/100
7/7 [==============================] - 0s 8ms/step - loss: 12.0719 - val_loss: 11.4693
Epoch 7/100
7/7 [==============================] - 0s 8ms/step - loss: 11.7063 - val_loss: 11.1841
Epoch 8/100
7/7 [==============================] - 0s 8ms/step - loss: 11.4549 - val_loss: 11.0009
Epoch 9/100
7/7 [==============================] - 0s 8ms/step - loss: 11.2974 - val_loss: 10.8895
Epoch 10/100
7/7 [==============================] - 0s 8ms/step - loss: 11.2018 - val_loss: 10.8220
Epoch 11

#Deep Learning With Keras
With its embedding layers this is similar to the matrix factorization approach above, but instead of using a fixed dot-product as recommendation we will utilize some dense layers so the network can find better combinations.

In [ ]:
type(test_user_data)

pandas.core.series.Series

In [ ]:
# Create user- & place-id mapping
user_id_mapping = {id:i for i, id in enumerate(data_filtered['User_Id'].unique())}
place_id_mapping = {id:i for i, id in enumerate(data_filtered['Place_Id'].unique())}


# Create correctly mapped train- & testset
train_user_data = df_train['User_Id'].map(user_id_mapping)
train_place_data = df_train['Place_Id'].map(place_id_mapping)

test_user_data = df_test['User_Id'].map(user_id_mapping)
test_place_data = df_test['Place_Id'].map(place_id_mapping)


# Get input variable-sizes
users = len(user_id_mapping)
places = len(place_id_mapping)
embedding_size = 10

# Setup variables
user_embedding_size = 20
place_embedding_size = 10


##### Create model
# Set input layers
user_id_input = Input(shape=[1], name='user')
place_id_input = Input(shape=[1], name='place')

# Create embedding layers for users and places
user_embedding = Embedding(output_dim=user_embedding_size, 
                           input_dim=users,
                           input_length=1, 
                           name='user_embedding')(user_id_input)
place_embedding = Embedding(output_dim=place_embedding_size, 
                            input_dim=places,
                            input_length=1, 
                            name='item_embedding')(place_id_input)

# Reshape the embedding layers
user_vector = Reshape([user_embedding_size])(user_embedding)
place_vector = Reshape([place_embedding_size])(place_embedding)

# Concatenate the reshaped embedding layers
concat = Concatenate()([user_vector, place_vector])

# Combine with dense layers
dense = Dense(256)(concat)
y = Dense(1)(dense)

# Setup model
model = Model(inputs=[user_id_input, place_id_input], outputs=y)
model.compile(
    loss = "mse",optimizer ="adam"
)



# Fit model
history = model.fit([train_user_data, train_place_data],
          df_train['Place_Rating'],
          batch_size=256, 
          epochs=100,
          validation_split=0.1,
          shuffle=True)

# Test model
y_pred = model.predict([test_user_data, test_place_data])
#y_pred = model.predict([user_place_array])

y_true = df_test['Place_Rating'].values

#  Compute RMSE
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Deep Learning: {:.4f} RMSE'.format(rmse))

NameError: ignored

In [ ]:
# Menyiapkan dataframe
place_df = place[['Place_Id','Place_Name','Category','Rating','Weekend Holiday Price']]
place_df.columns = ['id','place_name','category','rating','price']
df = rating.copy()

In [ ]:
user_id = df.User_Id.sample(1).iloc[0]
place_visited_by_user = df[df.User_Id == user_id]

In [ ]:
def dict_encoder(col, data=df):

  # Mengubah kolom suatu dataframe menjadi list tanpa nilai yang sama
  unique_val = data[col].unique().tolist()

  # Melakukan encoding value kolom suatu dataframe ke angka
  val_to_val_encoded = {x: i for i, x in enumerate(unique_val)}

  # Melakukan proses encoding angka ke value dari kolom suatu dataframe
  val_encoded_to_val = {i: x for i, x in enumerate(unique_val)}
  return val_to_val_encoded, val_encoded_to_val

In [ ]:
# Encoding Place_Id
place_to_place_encoded, place_encoded_to_place = dict_encoder('Place_Id')

# Mapping Place_Id ke dataframe place
df['place'] = df['Place_Id'].map(place_to_place_encoded)

In [ ]:
# Encoding User_Id
user_to_user_encoded, user_encoded_to_user = dict_encoder('User_Id')

# Mapping User_Id ke dataframe
df['user'] = df['User_Id'].map(user_to_user_encoded)

In [ ]:
# Mendapatkan jumlah user dan place
num_users, num_place = len(user_to_user_encoded), len(place_to_place_encoded)
 
# Mengubah rating menjadi nilai float
df['Place_Rating'] = df['Place_Rating'].values.astype(np.float32)
 
# Mendapatkan nilai minimum dan maksimum rating
min_rating, max_rating = min(df['Place_Rating']), max(df['Place_Rating'])
 
print(f'Number of User: {num_users}, Number of Place: {num_place}, Min Rating: {min_rating}, Max Rating: {max_rating}')

Number of User: 100, Number of Place: 75, Min Rating: 1.0, Max Rating: 5.0


In [ ]:
# Membuat data lokasi yang belum dikunjungi user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Place_Id.values)]['id'] 
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)
 
place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

In [ ]:
place_to_place_encoded

{1: 46,
 2: 0,
 3: 30,
 4: 47,
 5: 31,
 6: 32,
 7: 1,
 8: 2,
 9: 48,
 10: 33,
 11: 34,
 12: 3,
 13: 4,
 14: 72,
 15: 49,
 16: 58,
 17: 50,
 18: 35,
 19: 5,
 20: 51,
 21: 64,
 22: 36,
 23: 6,
 24: 65,
 25: 7,
 26: 69,
 27: 59,
 28: 70,
 29: 52,
 30: 37,
 31: 8,
 32: 38,
 33: 9,
 34: 66,
 35: 10,
 36: 11,
 37: 12,
 38: 13,
 39: 14,
 40: 15,
 41: 16,
 42: 60,
 43: 17,
 44: 53,
 45: 18,
 46: 19,
 47: 20,
 48: 21,
 49: 22,
 50: 23,
 51: 24,
 52: 25,
 53: 39,
 54: 61,
 55: 40,
 56: 54,
 57: 41,
 58: 26,
 59: 27,
 60: 55,
 61: 73,
 62: 74,
 63: 62,
 64: 42,
 65: 63,
 66: 71,
 67: 28,
 68: 29,
 69: 56,
 70: 43,
 71: 44,
 72: 45,
 73: 57,
 74: 67,
 75: 68}

In [ ]:
user_place_array
test_user_data = usersd.map(user_id_mapping)
test_place_data =placesd.map(place_id_mapping)

AttributeError: ignored

In [2]:
# Mengambil top 7 recommendation
ratings = model.predict(user_place_array,50).Flatten()
top_ratings_indices = ratings.argsort()[-7:][::-1]
recommended_place_ids = [
    place_encoded_to_place.get(place_not_visited[x][0]) for x in top_ratings_indices
]
 
print('Daftar rekomendasi untuk: {}'.format('User ' + str(user_id)))
print('===' * 15,'\n')
print('----' * 15)
print('Tempat dengan rating wisata paling tinggi dari user')
print('----' * 15)
 
top_place_user = (
    place_visited_by_user.sort_values(
        by = 'Place_Ratings',
        ascending=False
    )
    .head(5)
    .Place_Id.values
)
 
place_df_rows = place_df[place_df['id'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.place_name, ':', row.category)

print('')
print('----' * 15)
print('Top 7 place recommendation')
print('----' * 15)
 
recommended_place = place_df[place_df['id'].isin(recommended_place_ids)]
for row, i in zip(recommended_place.itertuples(), range(1,8)):
    print(i,'.', row.place_name, '\n    ', row.category, ',', 'Harga Tiket Masuk ', row.price, ',', 'Rating Wisata ', row.rating,'\n')

print('==='*15)

NameError: ignored